In [1]:
import ipywidgets as widgets
import pandas as pd
from ipywidgets import Layout, AppLayout

import techminer.analytics as tc
import techminer.plots as plt
from techminer.analytics import load_scopus
import techminer.dashboards as dash
from techminer.keywords import Keywords

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)

df = load_scopus(df)

2020-06-02 03:13:27,359 - INFO - Renaming and selecting columns ...
2020-06-02 03:13:33,267 - INFO - Formatting author names ...
2020-06-02 03:13:33,272 - INFO - Disambiguating author names ...
2020-06-02 03:13:33,330 - INFO - Removing part of titles in foreing languages ...
2020-06-02 03:13:33,333 - INFO - Fusioning author and index keywords ...
2020-06-02 03:13:33,345 - INFO - NumExpr defaulting to 8 threads.
2020-06-02 03:13:33,361 - INFO - Extracting countries from affiliations ...
2020-06-02 03:13:39,579 - INFO - Extracting institutions from affiliations ...
2020-06-02 03:13:39,590 - INFO - Extracting country of 1st author ...
2020-06-02 03:13:39,593 - INFO - Extracting affiliation of 1st author ...
2020-06-02 03:13:39,597 - INFO - Counting number of authors ...


In [2]:
COLORMAPS = [
    "Greys",
    "Purples",
    "Blues",
    "Greens",
    "Oranges",
    "Reds",
    "YlOrBr",
    "YlOrRd",
    "OrRd",
    "PuRd",
    "RdPu",
    "BuPu",
    "GnBu",
    "PuBu",
    "YlGnBu",
    "PuBuGn",
    "BuGn",
    "YlGn",
    "Pastel1",
    "Pastel2",
    "Paired",
    "Accent",
    "Dark2",
    "Set1",
    "Set2",
    "Set3",
    "tab10",
    "tab20",
    "tab20b",
    "tab20c",
]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st" "Institutions",
    "Keywords",
    "Source title",
]

In [3]:
def html_title(x):
    return (
        "<h1>{}</h1>".format(x)
        + "<hr style='height:2px;border-width:0;color:gray;background-color:gray'>"
    )

In [62]:
FIGSIZE = (17, 8.0)
LEFT_PANEL_HEIGHT = "588px"
PANE_HEIGHTS = ["80px", "650px", 0]
WIDGET_WIDTH = "200px"

In [67]:
def co_occurrence_analysis(x):
    
    def tab_0():
        #
        def compute(rows, columns, analysis_type, row_order, column_order, cmap, minmax):
            #
            
            if analysis_type == "Frequency":
                matrix, limit_value = tc.co_occurrence(
                    x,
                    rows,
                    columns,
                    as_matrix=True,
                    minmax=(selection_range.value[0], selection_range.value[1]),
                    keywords=None,
                    retmaxval=True,
                )
            else:
                matrix, limit_value = tc.co_citation(
                    x,
                    rows,
                    columns,
                    as_matrix=True,
                    minmax=(selection_range.value[0], selection_range.value[1]),
                    keywords=None,
                    retmaxval=True,
                )
            #
            values = selection_range.value
            if values[1] > limit_value or selection_range.max < limit_value:
                selection_range.max = limit_value
                if values[0] > limit_value:
                    selection_range.min = 0
                # selection_range.value = [0, limit_value]
            #
            output.clear_output()
            with output:
                if len(matrix.columns) < 21 and len(matrix.index) < 21:
                    display(matrix.style.background_gradient(cmap=cmap))
                else:
                    display(matrix)  # .style.background_gradient(cmap=cmap)


            
            
            
            
            
            
            
            
            plots = {"Heatmap": plt.heatmap, "Gant": plt.gant}
            plot = plots[plot_type]
            #
            if analysis_type == "Frequency":
                top = tc.documents_by_term(x, term).head(top_n)[term].tolist()
                matrix = tc.documents_by_term_per_year(x, term, as_matrix=True)
            else:
                top = tc.citations_by_term(x, term).head(top_n)[term].tolist()
                matrix = tc.citations_by_term_per_year(x, term, as_matrix=True)
            matrix = matrix[top]
            output.clear_output()
            with output:
                if plot_type == "Heatmap":
                    display(plot(matrix, cmap=cmap, figsize=FIGSIZE))
                if plot_type == "Gant":
                    display(plot(matrix, figsize=FIGSIZE))

        #
        LEFT_PANEL = [
            (
                "Term to analyze:",
                "term",
                widgets.Select(
                    options=[z for z in COLUMNS if z in x.columns],
                    ensure_option=True,
                    disabled=False,
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
            (
                "By:",
                "term",
                widgets.Select(
                    options=[z for z in COLUMNS if z in x.columns],
                    ensure_option=True,
                    disabled=False,
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
            (
                "Analysis type:",
                "analysis_type",
                widgets.Dropdown(
                    options=["Frequency", "Citation"],
                    value="Frequency",
                    disable=False,
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),

            
            
            
            
            (
                "Plot type:",
                "plot_type",
                widgets.Dropdown(
                    options=["Heatmap"],
                    disable=False,
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
            (
                "Colormap:",
                "cmap",
                widgets.Dropdown(
                    options=COLORMAPS, disable=False, layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
            (
                "Top N:",
                "top_n",
                widgets.IntSlider(
                    value=10,
                    min=10,
                    max=50,
                    step=1,
                    disabled=False,
                    continuous_update=False,
                    orientation="horizontal",
                    readout=True,
                    readout_format="d",
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
        ]
        #
        args = {key: value for _, key, value in LEFT_PANEL}
        output = widgets.Output()
        with output:
            display(widgets.interactive_output(compute, args,))
        return widgets.HBox(
            [
                widgets.VBox(
                    [
                        widgets.VBox([widgets.Label(value=text), widget])
                        for text, _, widget in LEFT_PANEL
                    ],
                    layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
                ),
                widgets.VBox([output]),
            ]
        )

    #
    #
    body = widgets.Tab()
    body.children = [tab_0()]
    #
    body.set_title(0, "Heatmap")

    return AppLayout(
        header=widgets.HTML(value=html_title("Summary by Term per Year")),
        left_sidebar=None,
        center=body,
        right_sidebar=None,
        pane_heights=PANE_HEIGHTS,
    )


    
    
    
    
    
    
    def tab_co_occurrence_heatmap():
        def compute_by_term(
            rows, columns, analysis_type, row_order, column_order, cmap, minmax
        ):
            #
            if analysis_type == "Frequency":
                matrix, limit_value = tc.co_occurrence(
                    x,
                    rows,
                    columns,
                    as_matrix=True,
                    minmax=(selection_range.value[0], selection_range.value[1]),
                    keywords=None,
                    retmaxval=True,
                )
            else:
                matrix, limit_value = tc.co_citation(
                    x,
                    rows,
                    columns,
                    as_matrix=True,
                    minmax=(selection_range.value[0], selection_range.value[1]),
                    keywords=None,
                    retmaxval=True,
                )
            #
            values = selection_range.value
            if values[1] > limit_value or selection_range.max < limit_value:
                selection_range.max = limit_value
                if values[0] > limit_value:
                    selection_range.min = 0
                # selection_range.value = [0, limit_value]
            #
            output.clear_output()
            with output:
                if len(matrix.columns) < 21 and len(matrix.index) < 21:
                    display(matrix.style.background_gradient(cmap=cmap))
                else:
                    display(matrix)  # .style.background_gradient(cmap=cmap)

                    
        ################        
        #
        rows = widgets.Select(
            options=[z for z in COLUMNS if z in x.columns],
            ensure_option=True,
            disabled=False,
        )
        columns = widgets.Select(
            options=[z for z in COLUMNS if z in x.columns],
            ensure_option=True,
            disabled=False,
        )
        analysis_type = widgets.Dropdown(
            options=["Frequency", "Citation"], value="Frequency", disable=False,
        )
        selection_range = widgets.IntRangeSlider(
            value=[0, 1000],
            min=0,
            max=1000,
            step=1,
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        row_order = widgets.Dropdown(
            options=["Alphabetic asc.", "Alphabetic desc.", "F/C asc.", "F/C desc."],
            value="F/C desc.",
            disable=False,
        )
        column_order = widgets.Dropdown(
            options=["Alphabetic asc.", "Alphabetic desc.", "F/C asc.", "F/C desc."],
            value="F/C desc.",
            disable=False,
        )
        cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
        #
        output = widgets.Output()
        with output:
            display(
                widgets.interactive_output(
                    compute_by_term,
                    {
                        "rows": rows,
                        "columns": columns,
                        "analysis_type": analysis_type,
                        "row_order": row_order,
                        "column_order": column_order,
                        "cmap": cmap,
                        "minmax": selection_range,
                    },
                )
            )
        #
        return widgets.HBox(
            [
                widgets.VBox(
                    [
                        widgets.VBox([widgets.Label(value="Rows:"), rows]),
                        widgets.VBox([widgets.Label(value="Columns:"), columns]),
                        widgets.VBox(
                            [widgets.Label(value="Analysis type:"), analysis_type]
                        ),
                        widgets.VBox([widgets.Label(value="Range:"), selection_range]),
                        widgets.VBox([widgets.Label(value="Row order:"), row_order]),
                        widgets.VBox(
                            [widgets.Label(value="Column order:"), column_order]
                        ),
                        widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
                    ],
                    layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
                ),
                widgets.VBox([output]),
            ]
        )

    #
    #
    tab_nest = widgets.Tab()
    tab_nest.children = [tab_co_occurrence_heatmap()]
    tab_nest.set_title(0, "Heatmap")

    return AppLayout(
        header=widgets.HTML(value=html_title("Co-occurrence Analysis")),
        left_sidebar=None,
        center=tab_nest,
        right_sidebar=None,
        pane_widths=PANE_WIDHTS,
        pane_heights=PANE_HEIGHTS,
    )


#
#
#
co_occurrence_analysis(df)




####################################


def summary_by_term_per_year(x):
    #


#
#
#
summary_by_term_per_year(df)

AppLayout(children=(HTML(value="<h1>Summary by Term per Year</h1><hr style='height:2px;border-width:0;color:gr…

In [ ]:
def correlation_analysis(x):
    def tab_heatmap():
        def compute_by_term(column, by, method, minmax, cmap, filter_type, top_n):
            #
            minmax = (minmax[0], minmax[1])
            #
            matrix, limit_values = tc.corr(
                x,
                column=column,
                by=by,
                method=method.lower(),
                show_between=minmax,
                cmap=cmap,
                filter_by=filter_type,
                top_n=top_n,
                as_matrix=True,
                get_minmax=True,
            )
            output.clear_output()
            with output:
                if len(matrix.columns) < 51 and len(matrix.index) < 51:
                    display(
                        matrix.style.format("{:.3f}").background_gradient(cmap=cmap)
                    )
                else:
                    display(matrix.style.format("{:.3f}"))

        #
        column = widgets.Select(
            options=[z for z in COLUMNS if z in x.columns],
            ensure_option=True,
            disabled=False,
        )
        by = widgets.Select(
            options=[z for z in COLUMNS if z in x.columns],
            ensure_option=True,
            disabled=False,
        )
        method = widgets.Dropdown(
            options=["Pearson", "Kendall", "Spearman"], value="Pearson", disable=False,
        )
        selection_range = widgets.FloatRangeSlider(
            value=[-1.0, 1.0],
            min=-1.0,
            max=1.0,
            step=0.1,
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="+.1f",
        )
        filter_type = widgets.Dropdown(
            options=["Frequency", "Citation"], value="Frequency", disable=False,
        )
        top_n = widgets.IntSlider(
            value=10,
            min=10,
            max=50,
            step=1,
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
        #
        output = widgets.Output()
        with output:
            display(
                widgets.interactive_output(
                    compute_by_term,
                    {
                        "column": column,
                        "by": by,
                        "method": method,
                        "minmax": selection_range,
                        "cmap": cmap,
                        "filter_type": filter_type,
                        "top_n": top_n,
                    },
                )
            )
        #
        return widgets.HBox(
            [
                widgets.VBox(
                    [
                        widgets.VBox([widgets.Label(value="Term:"), column]),
                        widgets.VBox([widgets.Label(value="By term:"), by]),
                        widgets.VBox([widgets.Label(value="Method:"), method]),
                        widgets.VBox(
                            [widgets.Label(value="Filter type:"), filter_type]
                        ),
                        widgets.VBox([widgets.Label(value="Top n:"), top_n]),
                        widgets.VBox([widgets.Label(value="Range:"), selection_range]),
                        widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
                    ],
                    layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
                ),
                widgets.VBox([output]),
            ]
        )

    #
    tabs = widgets.Tab()
    tabs.children = [tab_heatmap()]
    tabs.set_title(0, "Heatmap")

    return AppLayout(
        header=widgets.HTML(value=html_title("Correlation Analysis")),
        left_sidebar=None,
        center=tabs,
        right_sidebar=None,
        pane_widths=PANE_WIDHTS,
        pane_heights=PANE_HEIGHTS,
    )


#
#
#
correlation_analysis(df)

In [ ]:
def co_occurrence_analysis(x):
    def tab_co_occurrence_heatmap():
        def compute_by_term(
            rows, columns, analysis_type, row_order, column_order, cmap, minmax
        ):
            #
            if analysis_type == "Frequency":
                matrix, limit_value = tc.co_occurrence(
                    x,
                    rows,
                    columns,
                    as_matrix=True,
                    minmax=(selection_range.value[0], selection_range.value[1]),
                    keywords=None,
                    retmaxval=True,
                )
            else:
                matrix, limit_value = tc.co_citation(
                    x,
                    rows,
                    columns,
                    as_matrix=True,
                    minmax=(selection_range.value[0], selection_range.value[1]),
                    keywords=None,
                    retmaxval=True,
                )
            #
            values = selection_range.value
            if values[1] > limit_value or selection_range.max < limit_value:
                selection_range.max = limit_value
                if values[0] > limit_value:
                    selection_range.min = 0
                # selection_range.value = [0, limit_value]
            #
            output.clear_output()
            with output:
                if len(matrix.columns) < 21 and len(matrix.index) < 21:
                    display(matrix.style.background_gradient(cmap=cmap))
                else:
                    display(matrix)  # .style.background_gradient(cmap=cmap)

        #
        rows = widgets.Select(
            options=[z for z in COLUMNS if z in x.columns],
            ensure_option=True,
            disabled=False,
        )
        columns = widgets.Select(
            options=[z for z in COLUMNS if z in x.columns],
            ensure_option=True,
            disabled=False,
        )
        analysis_type = widgets.Dropdown(
            options=["Frequency", "Citation"], value="Frequency", disable=False,
        )
        selection_range = widgets.IntRangeSlider(
            value=[0, 1000],
            min=0,
            max=1000,
            step=1,
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            readout_format="d",
        )
        row_order = widgets.Dropdown(
            options=["Alphabetic asc.", "Alphabetic desc.", "F/C asc.", "F/C desc."],
            value="F/C desc.",
            disable=False,
        )
        column_order = widgets.Dropdown(
            options=["Alphabetic asc.", "Alphabetic desc.", "F/C asc.", "F/C desc."],
            value="F/C desc.",
            disable=False,
        )
        cmap = widgets.Dropdown(options=COLORMAPS, disable=False,)
        #
        output = widgets.Output()
        with output:
            display(
                widgets.interactive_output(
                    compute_by_term,
                    {
                        "rows": rows,
                        "columns": columns,
                        "analysis_type": analysis_type,
                        "row_order": row_order,
                        "column_order": column_order,
                        "cmap": cmap,
                        "minmax": selection_range,
                    },
                )
            )
        #
        return widgets.HBox(
            [
                widgets.VBox(
                    [
                        widgets.VBox([widgets.Label(value="Rows:"), rows]),
                        widgets.VBox([widgets.Label(value="Columns:"), columns]),
                        widgets.VBox(
                            [widgets.Label(value="Analysis type:"), analysis_type]
                        ),
                        widgets.VBox([widgets.Label(value="Range:"), selection_range]),
                        widgets.VBox([widgets.Label(value="Row order:"), row_order]),
                        widgets.VBox(
                            [widgets.Label(value="Column order:"), column_order]
                        ),
                        widgets.VBox([widgets.Label(value="Colormap:"), cmap]),
                    ],
                    layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
                ),
                widgets.VBox([output]),
            ]
        )

    #
    #
    tab_nest = widgets.Tab()
    tab_nest.children = [tab_co_occurrence_heatmap()]
    tab_nest.set_title(0, "Heatmap")

    return AppLayout(
        header=widgets.HTML(value=html_title("Co-occurrence Analysis")),
        left_sidebar=None,
        center=tab_nest,
        right_sidebar=None,
        pane_widths=PANE_WIDHTS,
        pane_heights=PANE_HEIGHTS,
    )


#
#
#
co_occurrence_analysis(df)

In [70]:
def summary_by_year(x):
    """ Summary by year dashboard.
    
    Args:
        df (pandas.DataFrame): bibliographic dataframe.
    
    """

    def tab_0():
        #
        def compute(selected_plot, plot_type, cmap):
            #
            plots = {"bar": plt.bar, "barh": plt.barh}
            data = {
                "Documents by Year": ["Year", "Num Documents"],
                "Cum. Documents by Year": ["Year", "Num Documents (Cum)"],
                "Times Cited by Year": ["Year", "Cited by"],
                "Cum. Times Cited by Year": ["Year", "Cited by (Cum)"],
                "Avg. Times Cited by Year": ["Year", "Avg. Cited by"],
            }
            #
            df = tc.summary_by_year(x)
            df = df[data[selected_plot]]
            plot = plots[plot_type]
            output.clear_output()
            with output:
                display(plot(df, cmap=cmap, figsize=FIGSIZE))

        #
        LEFT_PANEL = [
            (
                "Plot:",
                "selected_plot",
                widgets.Dropdown(
                    options=[
                        "Documents by Year",
                        "Cum. Documents by Year",
                        "Times Cited by Year",
                        "Cum. Times Cited by Year",
                        "Avg. Times Cited by Year",
                    ],
                    value="Documents by Year",
                    disable=False,
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
            (
                "Plot type:",
                "plot_type",
                widgets.Dropdown(
                    options=["bar", "barh"],
                    disable=False,
                    layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
            (
                "Colormap:",
                "cmap",
                widgets.Dropdown(
                    options=COLORMAPS, disable=False, layout=Layout(width=WIDGET_WIDTH),
                ),
            ),
        ]
        #
        args = {key: value for _, key, value in LEFT_PANEL}
        output = widgets.Output()
        with output:
            display(widgets.interactive_output(compute, args,))
        return widgets.HBox(
            [
                widgets.VBox(
                    [
                        widgets.VBox([widgets.Label(value=text), widget])
                        for text, _, widget in LEFT_PANEL
                    ],
                    layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
                ),
                widgets.VBox([output]),
            ]
        )

    #
    #
    body = widgets.Tab()
    body.children = [tab_0()]
    #
    body.set_title(0, "Time analysis")

    return AppLayout(
        header=widgets.HTML(value=html_title("Summary by Year")),
        left_sidebar=None,
        center=body,
        right_sidebar=None,
        pane_heights=PANE_HEIGHTS,
    )

summary_by_year(df)

AppLayout(children=(HTML(value="<h1>Summary by Year</h1><hr style='height:2px;border-width:0;color:gray;backgr…